In [1]:
# use the movie lens dataset to build a recommender system
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv('FileStore/tables/movielens_ratings.csv', inferSchema = True, header = True)

In [5]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [7]:
# train/test split
training, test = data.randomSplit([0.8, 0.2])

In [8]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol = 'rating')

In [9]:
model = als.fit(training)

In [10]:
prediction = model.transform(test)

In [11]:
prediction.show()
# rating is the real rating of the user towards the item, 
# prediction is the predicted rating
# look at the difference to know how good/bad the model is performing

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 1.0| 29| 1.3885988|
 31| 1.0| 0| 1.9936961|
 85| 3.0| 6| 3.622984|
 85| 1.0| 23|-0.30188492|
 85| 1.0| 25| 1.7524825|
 85| 1.0| 29| -0.0901113|
 65| 2.0| 5| 0.6629342|
 65| 1.0| 19| 2.47808|
 65| 1.0| 4| -0.4118066|
 65| 1.0| 2| 3.2304063|
 53| 1.0| 9| 3.2407756|
 78| 1.0| 1| 1.0939376|
 78| 1.0| 24| 1.071108|
 78| 1.0| 11|-0.48249894|
 34| 1.0| 15| 1.4920716|
 34| 1.0| 17| 2.6319313|
 34| 1.0| 4| 1.087449|
 81| 1.0| 16| 0.10839361|
 81| 2.0| 5| 0.3923496|
 81| 1.0| 19| 1.3173563|
+-------+------+------+-----------+
only showing top 20 rows

In [12]:
# evaluate the model formally
evaluator = RegressionEvaluator(metricName='rmse', labelCol = 'rating', predictionCol = 'prediction')

In [13]:
rmse = evaluator.evaluate(prediction)

In [14]:
print('RMSE')
print(rmse)
# star rating is from 1-5, the RMSE printed below is too big 
# considering the size of the data is not big enough
# its not that surprising that the model works so bad

RMSE
1.8013062269164153

In [15]:
# how to recommend a movie to a single user? 
single_user = test.filter(test['userid']==11).select(['movieId', 'userId'])

In [16]:
# predict the rating of the user of the movies he/she hasn't seen yet
# recommend the one with the highest predicted rating
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 0| 11|
 9| 11|
 12| 11|
 23| 11|
 40| 11|
 43| 11|
 71| 11|
 72| 11|
 75| 11|
 77| 11|
 78| 11|
+-------+------+

In [17]:

recommendations = model.transform(single_user)

In [18]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 0| 11| 1.5914747|
 40| 11| 1.1539488|
 9| 11| 1.1057937|
 23| 11| 1.1013994|
 72| 11| 0.93057233|
 78| 11|-0.48249894|
 75| 11| -0.8021713|
 77| 11| -1.3294479|
 71| 11| -1.5954291|
 12| 11| -2.5830784|
 43| 11| -4.305629|
+-------+------+-----------+

In [19]:
# cold start problem: users who haven't watched any movie on this platform
# could use survey: which movies do you like
# or: could ask which one typical customer profile are you most close to 
# but in general, the cold start problem is a challenge in recommender systems